In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
3.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
levers = copy.deepcopy(dike_model.levers)

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000020A6BAD5940>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000020A6BAD5940>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000020A6BAD5940>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000020A6BAD5940>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x0000020A6BAD5940>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x0000020A6BAD5940>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [8]:
from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

In [9]:
# running the model through EMA workbench
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policy = Policy("no policy", **get_do_nothing_dict())
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=10, policies = policy, uncertainty_sampling = Samplers.SOBOL)

[MainProcess/INFO] pool started with 12 workers
C:\Users\rvrij\anaconda\envs\Model_based_decisions\Lib\site-packages\scipy\stats\_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
[MainProcess/INFO] performing 400 scenarios * 1 policies * 1 model(s) = 400 experiments
100%|████████████████████████████████████████| 400/400 [01:52<00:00,  3.55it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
from ema_workbench import save_results
save_results(results, '10 scenarios SOBOL no policy.tar.gz')


[MainProcess/INFO] results saved successfully to C:\Users\rvrij\OneDrive\Documenten\Repositories Github\epa1361_open\final assignment\10 scenarios SOBOL no policy.tar.gz


In [10]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
print(outcome)


dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])
ScalarOutcome('Expected Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x0000020A6BAD5940>)


In [11]:
# only works because we have scalar outcomes
data = pd.DataFrame(outcomes)
print(data)

     A.1 Total Costs  A.1_Expected Number of Deaths  A.2 Total Costs  \
0       1.647128e+09                       1.506811     0.000000e+00   
1       1.627364e+09                       1.484247     0.000000e+00   
2       1.647128e+09                       1.506811     0.000000e+00   
3       1.647128e+09                       1.506811     0.000000e+00   
4       1.647128e+09                       1.506811     0.000000e+00   
..               ...                            ...              ...   
395     2.655744e+08                       0.167714     2.556991e+08   
396     2.246536e+08                       0.167714     2.162999e+08   
397     2.655744e+08                       0.167714     2.556991e+08   
398     2.246536e+08                       0.167714     2.162999e+08   
399     2.655744e+08                       0.167714     2.556991e+08   

     A.2_Expected Number of Deaths  A.3 Total Costs  \
0                         0.000000     6.166841e+06   
1                        

In [25]:
strings = ["_Expected Number of Deaths", " Total Costs"]
for i in range(5):
    print(f"outcome for A.{i+1}{strings[1]}")
    y = outcomes[f'A.{i+1}{strings[1]}']
    problem = get_SALib_problem(dike_model.uncertainties)
    Si = sobol.analyze(problem, y, calc_second_order=True, print_to_console=True)

outcome for A.1 Total Costs
                               ST   ST_conf
A.0_ID flood wave shape  0.001957  0.000926
A.1_Bmax                 0.000000  0.000000
A.1_Brate                0.000000  0.000000
A.1_pfail                0.987170  0.076684
A.2_Bmax                 0.000000  0.000000
A.2_Brate                0.000000  0.000000
A.2_pfail                0.000000  0.000000
A.3_Bmax                 0.000000  0.000000
A.3_Brate                0.000000  0.000000
A.3_pfail                0.000000  0.000000
A.4_Bmax                 0.000000  0.000000
A.4_Brate                0.000000  0.000000
A.4_pfail                0.000000  0.000000
A.5_Bmax                 0.000000  0.000000
A.5_Brate                0.000000  0.000000
A.5_pfail                0.000000  0.000000
discount rate 0          0.012689  0.001654
discount rate 1          0.012628  0.001612
discount rate 2          0.012767  0.001764
                               S1   S1_conf
A.0_ID flood wave shape  0.001183  0.003797
A.1_

In [12]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [13]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [01:19<00:00,  3.77it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [14]:
experiments, outcomes = results

In [15]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,7.744753e+07,0.094950,0.000000e+00,0.000000,8.428879e+07,0.049126,0.000000e+00,0.000000,253800000.0,0.0
1,5.397251e+07,0.000000,7.728817e+08,0.838196,0.000000e+00,0.000000,9.436089e+06,0.004566,0.000000e+00,0.000000,253800000.0,0.0
2,5.397251e+07,0.000000,1.119432e+08,0.094947,3.977018e+08,0.627378,0.000000e+00,0.000000,9.423593e+06,0.007504,253800000.0,0.0
3,5.397251e+07,0.000000,3.023717e+07,0.028025,1.205602e+09,1.958728,1.166847e+08,0.044614,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,1.612180e+08,0.128669,6.102318e+08,0.912002,1.261425e+08,0.046691,1.608439e+07,0.012223,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.000000e+00,0.000000,4.741683e+06,0.004141,3.767677e+07,0.014313,1.183023e+07,0.005463,4.310499e+08,0.334611,369700000.0,0.0
296,2.100464e+09,1.337864,8.517290e+08,0.689269,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
297,0.000000e+00,0.000000,1.179474e+08,0.087243,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
298,0.000000e+00,0.000000,2.949653e+07,0.026722,2.879840e+07,0.000000,6.479808e+07,0.027942,0.000000e+00,0.000000,369700000.0,0.0
